# EBAC - Regressão II - regressão múltipla

## Tarefa I

In [3]:
import numpy as np
import pandas as pd

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('previsao_de_renda.csv')
df.head(5)

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [10]:
# 1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.

# Convertendo variáveis qualitativas em dummies
data_encoded = pd.get_dummies(df, drop_first=True)

In [11]:
# Ajustando o modelo

y, X = patsy.dmatrices('''np.log(renda) ~ sexo 
                                                    + posse_de_veiculo 
                                                    + posse_de_imovel 
                                                    + C(qtd_filhos, Treatment(0))
                                                    + C(tipo_renda, Treatment('Assalariado'))
                                                    + C(educacao, Treatment('Secundário')) 
                                                    + C(estado_civil, Treatment('Casado')) 
                                                    + C(tipo_residencia, Treatment('Casa')) 
                                                    + idade 
                                                    + tempo_emprego 
                                                    + C(qt_pessoas_residencia, Treatment(2))
                                    ''', 
                       data=df)

sm.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     203.5
Date:                Mon, 05 Feb 2024   Prob (F-statistic):               0.00
Time:                        10:01:04   Log-Likelihood:                -13559.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12392   BIC:                         2.745e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.0932      0.039    183.905      0.000       7.018       7.169
sexo[T.M]                                                       0.7878      0.015     53.664      0.000       0.759       0.817
posse_de_veiculo[T.True]                                        0.0436      0.014      3.079      0.002       0.016       0.071
posse_de_imovel[T.True]                                         0.0828      0.014      5.907      0.000       0.055       0.110
C(qtd_filhos, Treatment(0))[T.1]                               -0.2501      0.112     -2.239      0.025      -0.469      -0.031
C(qtd_filhos, Treatment(0))[T.2]                               -0.5697      0.223     -2.559      0.011      -1.006      -0.133
C(qtd_filhos, Treatment(0))[T.3]                               -0.5560      0.486     -1.144      0.253      -1.509       0.397
C(qtd_filhos, Treatment(0))[T.4]                                0.2091      0.429      0.488      0.626      -0.631       1.049
C(qtd_filhos, Treatment(0))[T.5]                                0.1385      0.410      0.338      0.736      -0.666       0.943
C(qtd_filhos, Treatment(0))[T.7]                                0.0221      0.162      0.137      0.891      -0.295       0.339
C(qtd_filhos, Treatment(0))[T.14]                              -0.4630      0.261     -1.772      0.076      -0.975       0.049
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.2259      0.241      0.937      0.349      -0.247       0.699
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1547      0.015     10.359      0.000       0.125       0.184
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -0.2891      0.243     -1.190      0.234      -0.765       0.187
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0551      0.022      2.476      0.013       0.011       0.099
C(educacao, Treatment('Secundário'))[T.Primário]                0.0130      0.072      0.181      0.856      -0.128       0.154
C(educacao, Treatment('Secundário'))[T.Pós graduação]           0.1168      0.142      0.821      0.412      -0.162       0.396
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1069      0.014      7.678      0.000       0.080       0.134
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.0290      0.032     -0.897      0.370      -0.092       0.034
C(estado_civil, Treatment('Casado'))[T.Separado]                0.3176      0.112      2.831      0.005       0.098       0.537
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2556      0.110      2.319      0.020       0.040       0.472
C(estado_ci

É possível extrair, de forma simplificada, algumas conclusões:

. Sexo e posse de veículo e imóvel: Indivíduos do sexo masculino tendem a ter uma renda mais alta, e possuir veículo e imóvel está associado a uma renda maior.

. Número de filhos: Em geral, ter mais filhos está associado a uma renda menor, embora a significância estatística varie entre as diferentes categorias de número de filhos.

. Tipo de renda: Certos tipos de renda, como ser empresário ou servidor público, estão associados a uma renda mais alta em comparação com o tipo de renda padrão, que é ser assalariado.

. Educação e estado civil: Ter um grau de instrução mais alto e certos estados civis, como ser viúvo, estão associados a uma renda mais alta.

. Tipo de residência: O tipo de residência também influencia a renda, embora nem todos os coeficientes sejam significantes.

. Idade e tempo de emprego: Ambos estão positivamente associados à renda, o que é esperado, pois experiência e tempo de trabalho geralmente estão relacionados a salários mais altos

In [13]:
# 2. Remova a variável menos significante, no caso "C(educacao, Treatment('Secundário'))[T.Primário]"  e analise:

y, X = patsy.dmatrices('''np.log(renda) ~ sexo 
                                          + posse_de_veiculo 
                                          + posse_de_imovel 
                                          + C(qtd_filhos, Treatment(0))
                                          + C(tipo_renda, Treatment('Assalariado'))
                                          + C(educacao, Treatment('Secundário')) 
                                          + C(estado_civil, Treatment('Casado')) 
                                          + C(tipo_residencia, Treatment('Casa')) 
                                          + idade 
                                          + tempo_emprego 
                                          + C(qt_pessoas_residencia, Treatment(2))
                            ''', 
                       data=df)


sm.OLS(y, X).fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     203.5
Date:                Mon, 05 Feb 2024   Prob (F-statistic):               0.00
Time:                        10:16:37   Log-Likelihood:                -13559.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12392   BIC:                         2.745e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.0932      0.039    183.905      0.000       7.018       7.169
sexo[T.M]                                                       0.7878      0.015     53.664      0.000       0.759       0.817
posse_de_veiculo[T.True]                                        0.0436      0.014      3.079      0.002       0.016       0.071
posse_de_imovel[T.True]                                         0.0828      0.014      5.907      0.000       0.055       0.110
C(qtd_filhos, Treatment(0))[T.1]                               -0.2501      0.112     -2.239      0.025      -0.469      -0.031
C(qtd_filhos, Treatment(0))[T.2]                               -0.5697      0.223     -2.559      0.011      -1.006      -0.133
C(qtd_filhos, Treatment(0))[T.3]                               -0.5560      0.486     -1.144      0.253      -1.509       0.397
C(qtd_filhos, Treatment(0))[T.4]                                0.2091      0.429      0.488      0.626      -0.631       1.049
C(qtd_filhos, Treatment(0))[T.5]                                0.1385      0.410      0.338      0.736      -0.666       0.943
C(qtd_filhos, Treatment(0))[T.7]                                0.0221      0.162      0.137      0.891      -0.295       0.339
C(qtd_filhos, Treatment(0))[T.14]                              -0.4630      0.261     -1.772      0.076      -0.975       0.049
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.2259      0.241      0.937      0.349      -0.247       0.699
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1547      0.015     10.359      0.000       0.125       0.184
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -0.2891      0.243     -1.190      0.234      -0.765       0.187
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0551      0.022      2.476      0.013       0.011       0.099
C(educacao, Treatment('Secundário'))[T.Primário]                0.0130      0.072      0.181      0.856      -0.128       0.154
C(educacao, Treatment('Secundário'))[T.Pós graduação]           0.1168      0.142      0.821      0.412      -0.162       0.396
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1069      0.014      7.678      0.000       0.080       0.134
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.0290      0.032     -0.897      0.370      -0.092       0.034
C(estado_civil, Treatment('Casado'))[T.Separado]                0.3176      0.112      2.831      0.005       0.098       0.537
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2556      0.110      2.319      0.020       0.040       0.472
C(estado_ci

Analisando os resultados, podemos ver que não houve mudanças significativas nos valores de R-quadrado, R-quadrado ajustado ou estatística F. Isso indica que, mesmo após remover a variável menos significativa, o ajuste do modelo permaneceu essencialmente o mesmo. 

In [14]:
# 3. Siga removendo as variáveis menos significantes, sempre que o p-value for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.

smf.ols(formula='''
                np.log(renda) ~ sexo 
                                + posse_de_veiculo 
                                + posse_de_imovel
                                + tipo_renda == 'Empresário'
                                + tipo_renda == 'Servidor público'
                                + educacao == 'Superior completo'
                                + qt_pessoas_residencia == 3
                                + qt_pessoas_residencia == 4
                                + idade 
                                + tempo_emprego
                ''', 
        data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     688.2
Date:                Mon, 05 Feb 2024   Prob (F-statistic):               0.00
Time:                        10:21:01   Log-Likelihood:                -13576.
No. Observations:               12427   AIC:                         2.717e+04
Df Residuals:                   12416   BIC:                         2.726e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                    7.0815      0.035    201.800      0.000       7.013       7.150
sexo[T.M]                                    0.7844      0.015     53.961      0.000       0.756       0.813
posse_de_veiculo[T.True]                     0.0437      0.014      3.117      0.002       0.016       0.071
posse_de_imovel[T.True]                      0.0824      0.014      6.011      0.000       0.055       0.109
tipo_renda == 'Empresário'[T.True]           0.1534      0.015     10.318      0.000       0.124       0.182
tipo_renda == 'Servidor público'[T.True]     0.0572      0.022      2.578      0.010       0.014       0.101
educacao == 'Superior completo'[T.True]      0.1083      0.014      7.979      0.000       0.082       0.135
qt_pessoas_residencia == 3[T.True]           0.0443      0.017      2.596      0.009       0.011       0.078
qt_pessoas_residencia == 4[T.True]           0.0843      0.022      3.806      0.000       0.041       0.128
idade                                        0.0060      0.001      7.783      0.000       0.004       0.008
tempo_emprego                                0.0615      0.001     59.297      0.000       0.059       0.064
==============================================================================
Omnibus:                        0.888   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.641   Jarque-Bera (JB):                0.862
Skew:                           0.018   Prob(JB):                        0.650
Kurtosis:                       3.018   Cond. No.                         241.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Mesmo retirando variáveis menos sifgnificativas não houve nenhuma melhora ou piora relevante.